In [144]:
import sys
sys.path.append("../src") 
sys.path.append("../data") 
import SDE_class as sde
import importlib
importlib.reload(sde)
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import util as util
importlib.reload(util)
import math
from scipy.stats import norm

# 独立増分の確認

$$X_t-X_s \amalg X_s$$
$$X_t-X_s \sim N(0,t-s)$$

In [145]:
term=1
step=0.01
div=term/step
init=np.array([0.])
repeat_time=10000#pass本数
S=0

In [146]:
mymodel = sde.SDE_Markov(mymat=np.array([[0.]]), myvar =np.array([[1.]]),myinit=np.array([1.]),myscale=1.,myterm=term,step_size=step)

In [147]:
time_s=0.3
time_t=0.83

In [148]:
time_t_box=np.zeros(repeat_time)
time_s_box=np.zeros(repeat_time)
time_ts_box=np.zeros(repeat_time)

In [149]:
for k in range(repeat_time):
    many_step = mymodel.many_step(now_position=init)
    time_t_box[k]=many_step[0,time_t*div]
    time_s_box[k]=many_step[0,time_s*div]
    time_ts_box[k]=many_step[0,time_t*div]-many_step[0,time_s*div]
var_s=np.var(time_s_box)
var_ts=np.var(time_ts_box)
print(var_s)
print(var_ts)

/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/tamamoon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.291836739753
0.521622545708


In [150]:
mean_s=np.mean(time_s_box)
mean_ts=np.mean(time_ts_box)
print(mean_s)
print(mean_ts)

0.00132252684671
0.0065760201579


# brown motion を確認するための正規分布 であるかの検定
アンダーソン　ダーリング検定を用いる

方法
統計量 $A^2=-n-S $

nは標本数、$X_{(i)}$は標本を小さい順に並び替えたもの

$$S=\sum_{i=1}^n \frac{2i-1}{n}\bigl\{\log(F(X_{(i)}))+\log(1-F(X_{(n+1-i)}))\bigr\}$$

この統計量とCV値で比較して検定を行う。

有意水準　CV値

$15\% \cdots 1.610$

$10\% \cdots 1.933$

$5\% \cdots 2.492$

$2.5\% \cdots 3.070$

$1.0\% \cdots 3.857$

まずは確率変数を正規化したあと、

$S$を計算するために確率変数を小さい順にソートする。

今$n$は標本数なので$n=$repeat time

In [151]:
time_ts_box.sort()
time_ts_box.reshape(1,repeat_time)
#plt.hist(time_ts_box,50)
#standard_normal=time_ts_box.reshape(1,repeat_time)/np.sqrt(time_t-time_s)

array([[-2.53563768, -2.5177847 , -2.4081139 , ...,  2.35143725,
         2.53877826,  3.09853606]])

In [152]:
cdf_box=np.zeros(repeat_time)
for k in range(repeat_time):
    cdf_box[k]=norm.cdf(time_ts_box[k],0,np.sqrt(time_t-time_s))

In [153]:
static_box=np.zeros(repeat_time)
for k in range (repeat_time):
    static_box[k]=(2*(k+1)-1)/repeat_time*(np.log(cdf_box[k])+np.log(1-cdf_box[repeat_time-1-k]))

In [154]:
A=-repeat_time-sum(static_box)
A

0.78116070561372908

In [155]:
np.mean(static_box)

-1.0000781160705552

In [156]:
np.var(static_box)

0.18579398495572241

In [157]:
np.var(static_box,ddof=1)

0.18581256621234366

In [104]:
number=int(repeat_time)
S=0
for k in range(number):
    S=S+(2*(k+1)-1)/number*(np.log(norm.cdf(x=time_ts_box[k], loc=0, scale=np.sqrt(time_t-time_s)))+np.log(1-norm.cdf(x=time_ts_box[repeat_time-1-k], loc=0, scale=np.sqrt(time_t-time_s))))              
A=-repeat_time-S
print(A)

0.274302692716
